In [19]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import *

from sklearn.model_selection import train_test_split,StratifiedShuffleSplit,KFold,GroupKFold,StratifiedGroupKFold,GridSearchCV,RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,StackingClassifier,BaggingClassifier
from xgboost import XGBClassifier

In [2]:
dataTrain=pd.read_csv("data/Train.txt",sep="#|;",engine="python",header=None,index_col = False)
dataTest=pd.read_csv("data/Test.txt",sep="#|;",engine="python",header=None,index_col = False)
dataCros=pd.read_csv("data/Cross_Validation.txt",sep="#|;",engine="python",header=None,index_col = False)

data=pd.concat([dataTrain,dataTest,dataCros])

In [3]:
data.describe()

,0,1,2,3,4,5,6,7,8,9,...,537,538,539,540,541,542,543,544,545,546
count,20189.000000,20189.000000,20189.000000,20189.000000,20189.000000,20189.000000,20189.000000,20189.000000,20189.000000,20189.000000,...,20189.000000,20189.000000,20189.000000,20189.000000,20189.000000,20189.000000,20189.000000,20189.000000,20189.000000,20189.000000
mean,11.224479,-3.423828,-1.005713,1.706400,-1.159582,-0.855778,-1.103048,-0.267441,-0.128349,0.159302,...,-0.303256,0.773173,-0.236877,0.099867,0.113846,0.347402,0.046918,0.146214,0.026270,4.104033
std,6.941206,12.906586,6.641375,5.304308,4.364610,3.429600,2.600222,2.251118,1.959483,1.809170,...,3.432375,2.807844,2.592954,2.329421,1.958721,1.758277,1.606191,1.455374,1.358208,1.047831
min,0.000000,-41.387033,-25.716292,-16.649619,-16.727334,-14.097185,-11.150499,-10.332489,-9.643177,-9.146440,...,-15.739322,-12.423641,-10.907989,-14.311366,-9.523313,-8.734160,-7.556286,-5.533654,-8.683989,-0.528020
25%,5.000000,-13.156473,-5.589819,-2.133137,-3.958086,-3.075181,-2.867972,-1.727710,-1.367883,-0.974871,...,-2.618182,-1.118800,-1.925435,-1.350946,-1.086751,-0.725954,-0.952058,-0.771705,-0.825883,3.514669
50%,11.000000,-1.369778,-1.044129,1.768324,-0.981802,-0.777229,-1.086701,-0.218771,-0.067052,0.209278,...,-0.208494,0.889173,-0.080375,0.239705,0.201000,0.377365,0.078007,0.169320,0.056811,4.168822
75%,17.000000,6.766844,3.764928,5.421992,1.693227,1.461983,0.679469,1.242841,1.138865,1.347726,...,2.165309,2.715580,1.540627,1.672862,1.382336,1.474741,1.087508,1.099771,0.911312,4.761209
max,23.000000,28.907357,19.357245,21.135432,20.982353,11.971851,14.983369,10.504401,9.182450,8.129699,...,15.025263,10.739319,11.194952,9.769175,11.536425,7.482802,7.454840,6.677344,5.998552,7.183624


In [4]:
scaler=StandardScaler()
X=data.drop(0,axis=1)
X=scaler.fit_transform(X)
Y=data[0]

In [5]:
%%time
def find_best_model(learning_algorithm,X,Y,expected_accuracy=0.75,model_selection=None,model_selection_params={},params={}): 
    while True:
        for (train_size,rand) in zip(np.random.random(50),np.arange(1,50)):
            train_size=round((train_size+0.1)*0.8,2)
            X_train,X_test,y_train,y_test = train_test_split(X,Y,train_size=train_size,random_state=rand)
            
            model = learning_algorithm()
            #If params is not empty adding default parametrs
            for (key,value) in params.items():
                if hasattr(value,"__iter__"):
                    l=list(value)
                else:
                    l=[value]
                l.append(model.__dict__.get(key))
                params[key]=l
            
            if model_selection:
                model_selection = model_selection(model,params,scoring="accuracy",**model_selection_params)
                model = model_selection.fit(X_train,y_train)
            else:
                model.fit(X_train,y_train)

            X_val,X_test,y_val,y_test=train_test_split(X_test,y_test,train_size=0.50,random_state=1)

            y_val_predict=model.predict(X_val)
            y_test_predict=model.predict(X_test)

            val_accuracy = accuracy_score(y_val_predict,y_val)
            test_accuracy=accuracy_score(y_test_predict,y_test)

            print((train_size,rand),f"Model: {Model.__name__}, Validation Accuracy:{val_accuracy:.3}, Test Accuracy:{test_accuracy:.3}")
            if val_accuracy>=expected_accuracy and test_accuracy>=expected_accuracy:
                return test_accuracy,model

accuracy,model=find_best_model(DecisionTreeClassifier,X,Y,model_selection=None)

(0.44, 1) Model: DecisionTreeClassifier, Validation Accuracy:0.736, Test Accuracy:0.728
(0.09, 2) Model: DecisionTreeClassifier, Validation Accuracy:0.602, Test Accuracy:0.609
(0.21, 3) Model: DecisionTreeClassifier, Validation Accuracy:0.686, Test Accuracy:0.68
(0.68, 4) Model: DecisionTreeClassifier, Validation Accuracy:0.759, Test Accuracy:0.765
CPU times: user 27.3 s, sys: 170 ms, total: 27.5 s
Wall time: 27.5 s


In [6]:
accuracy,model=find_best_model(LogisticRegression,X,Y)

(0.12, 1) Model: LogisticRegression, Validation Accuracy:0.892, Test Accuracy:0.895


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [8]:
accuracy,model=find_best_model(RandomForestClassifier,X,Y,expected_accuracy=0.97)

(0.2, 1) Model: RandomForestClassifier, Validation Accuracy:0.934, Test Accuracy:0.937
(0.71, 2) Model: RandomForestClassifier, Validation Accuracy:0.97, Test Accuracy:0.97
(0.75, 3) Model: RandomForestClassifier, Validation Accuracy:0.969, Test Accuracy:0.975
(0.84, 4) Model: RandomForestClassifier, Validation Accuracy:0.976, Test Accuracy:0.972


In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,Y,train_size=0.66)          
model = LogisticRegression()
model.fit(X_train,y_train)
X_val,X_test,y_val,y_test=train_test_split(X_test,y_test,train_size=0.70,random_state=1)
y_val_predict=model.predict(X_val)
y_test_predict=model.predict(X_test)
val_accuracy = accuracy_score(y_val_predict,y_val)
test_accuracy=accuracy_score(y_test_predict,y_test)
print(f"Validation Accuracy:{val_accuracy:.3}, Test Accuracy:{test_accuracy:.3}")

In [ ]:
find_best_model(BaggingClassifier,X,Y,expected_accuracy=0.8)

In [ ]:
find_best_model(XGBClassifier,X,Y,expected_accuracy=0.96)

(0.22, 1) Model: XGBClassifier, Validation Accuracy:0.914, Test Accuracy:0.922
(0.35, 2) Model: XGBClassifier, Validation Accuracy:0.937, Test Accuracy:0.936


In [ ]:
GridSearchCV()

In [ ]:
d.update({"a":5})

In [ ]:
DecisionTreeClassifier.__name__

In [ ]:
estimator,
    param_grid,
    *,
    scoring=None,
    n_jobs=None,
    refit=True,
    cv=None,
    verbose=0,
    pre_dispatch='2*n_jobs',
    error_score=nan,
    return_train_score=False,
)

In [ ]:
{**DecisionTreeClassifier().__dict__,'criterion': 'entropy'}

In [ ]:
RandomizedSearchCV()

In [ ]:
model = DecisionTreeClassifier()
params = {"random_state": 2}
for (key,value) in params.items():
    if hasattr(value,"__iter__"):
        l=list(value)
    else:
        l=[value]
    l.append(model.__dict__.get(key))
    params[key]=l
                
                

In [ ]:
DecisionTreeClassifier(**{})

In [ ]:
dir(())

In [ ]:
hasattr([],"__iter__")

In [ ]:
print(model.__dict__.get("estimator5"))

In [ ]:
DecisionTreeClassifier().__dict__

In [15]:
set(Y.value_counts().index)

{0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23}

In [18]:
import threading

threading.Thread.run(find_best_model(BaggingClassifier,X,Y,expected_accuracy=0.8))

(0.35, 1) Model: BaggingClassifier, Validation Accuracy:0.874, Test Accuracy:0.872


AttributeError: 'tuple' object has no attribute '_target'

In [ ]:
print("sd")